In [1]:
import os
import requests
from bs4 import BeautifulSoup
import json
from lxml import etree
import logging
import time
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

## Come installare la cartella con tutti i documenti

1.   Vai su Google Drive.
2.   Trova la cartella condivisa "progettoIngDati".
3.   Fai clic con il pulsante destro del mouse sulla cartella e   
     seleziona :
     - Organizza.
     - Aggiungi Scorciatoia
     - Tutte le posizioni
4.   Scegli "My Drive" come destinazione per il collegamento.

In [5]:
#in questo modo caricate il vostro drive, che peró noi non vedremo!
#ma prima é importante che eseguiate le istruzioni qui sopra!
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)
SOURCES_DIR = 'sources/'
EXTRACTION_DIR = 'extraction/'

if not os.path.exists(SOURCES_DIR):
    os.makedirs(f'./{SOURCES_DIR}', exist_ok=True)

if not os.path.exists(EXTRACTION_DIR):
    os.makedirs(f'./{EXTRACTION_DIR}', exist_ok=True)

Cancella la cartella articoli/ (se  serve)

In [11]:
!rm -rf sources/

# DOI Scouting (article id) and HTML Download
Questa sezione esegue una query su arxiv.org cercando articoli che parlano di **LLM multi agent** e attraverso *bs4* ottiene i vari [DOI](https://en.wikipedia.org/wiki/Digital_object_identifier).
Con questi DOI è dunque possibile eseguire richieste specifiche per scaricare i documenti HTML del topic scelto.  

In [6]:
article_ids = set()
TOPIC = "LLM multi agent"
found_urls = set()

Scarica il file HTML5 da ar5iv

In [13]:
def get_ar5iv_html(article_id):
    urls = [ f"https://ar5iv.labs.arxiv.org/html/{article_id}",
             f"https://arxiv.org/html/{article_id}" ]               # sometimes html is provided on this url when it's not showing on ar5ive
    for url in urls:
        try:
            response = requests.get(url, timeout=10)
            logging.info(f"Request URL: {response.url}")

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')

                # Trova il tag <article> o altri contenuti rilevanti
                article = soup.find('article')

                if article:
                    found_urls.add(url)
                    return article

                # Se non troviamo un tag <article>, proviamo con <div> o <section>
                # if not article:
                #     article = soup.find('div', {'class': 'ltx_page'})  # Prova a trovare un'altra sezione

            else:
                logging.error(f"Error fetching article {article_id}, status code: {response.status_code}")
                return None

        except requests.exceptions.RequestException as e:
            logging.error(f"Error downloading article {article_id}: {e}")
            return None
    with open('/content/drive/MyDrive/progettoIngDati/read.me', 'w') as f:
        f.writelines([url + '\n' for url in found_urls])
        logging.info(f"Successfully wrote {len(found_urls)} urls to read.me")

Salva l'articolo in un file


In [14]:
def save_to_file(article, article_id):
    # Crea una cartella per gli articoli se non esiste
    #os.makedirs("articoli", exist_ok=True)
    #file_path = os.path.join("articoli", f"ar5iv_article_{article_id}.html")
    #with open(file_path, "w", encoding="utf-8") as file:
    #    file.write(article.prettify())
    #se hai montato la cartella questo funziona, altrimenti usa la parte commentata qui sopra
    file_path = os.path.join(SOURCES_DIR, f"ar5iv_article_{article_id}.html") #join with articoli_path
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(article.prettify())
    logging.info(f"Articolo {article_id} salvato.")

Cerca articoli simili e prova a scaricarne il contenuto html

In [15]:
def retrieve_article_id(url, params):
    response = requests.get(url, params=params)
    logging.info(f"Request URL: {response.url}")

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")

        ol_tag = soup.find('ol')
        if ol_tag:
            li_tags = ol_tag.find_all('li')
            local_article_counter = 0

            for li in li_tags:
                a_tag = li.find('a')
                if a_tag:
                    article_id = a_tag.text.split(':')[-1]
                    logging.info(f"Processing article number: {len(article_ids)}: {article_id}")
                    article = get_ar5iv_html(article_id)
                    if article:
                        logging.info(f"Article found: {article_id}")
                        local_article_counter += 1
                        article_ids.add(article_id)
                        save_to_file(article, article_id)
                    else:
                        logging.warning(f"Article {article_id} not found, skipping...")
                else:
                    logging.warning("No <a> element found in the <li> element.")

            logging.info(f"Number of local articles found: {local_article_counter} over 200")  # inside a single page
            time.sleep(2)
            return local_article_counter
        else:
            logging.warning("No <ol> element found.")
            return None
    else:
        logging.error(f"Failed to retrieve data. Status code: {response.status_code}")
        return None

Punto di lancio del codice, la query è composta da `url + params`

In [ ]:
# Define the URL for the advanced search on arXiv
url = "https://arxiv.org/search/advanced"

# Define the search parameters
params = {
    "advanced": "",
    "terms-0-term": TOPIC,              # Search term
    "terms-0-operator": "AND",          # Operator
    "terms-0-field": "all",             # Search field (all)
    "classification-computer_science": "y",  # Limit to computer science
    "classification-physics_archives": "all",  # Include all physics archives
    "classification-include_cross_list": "include",  # Include cross-list
    "date-filter_by": "all_dates",      # Filter by date (all dates)
    "date-year": "",                    # No specific year
    "date-from_date": "",               # No start date
    "date-to_date": "",                 # No end date
    "date-date_type": "submitted_date", # Search by submission date
    "abstracts": "hide",                # Show abstracts
    "size": 200,                        # Number of results to display
    "order": "submitted_date"           # Order by submission date (oldest first) #older documents are more likely to have html version
}

doi_retrieved = 0
while doi_retrieved <= 300:
    doi_retrieved += retrieve_article_id(url, params)
    params["start"] = doi_retrieved

logging.info(f"Number of articles retrieved: {len(article_ids)}")

# Data Extraction

In [92]:
def estrai_dati_da_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    parser = etree.HTMLParser()
    tree = etree.fromstring(content, parser)

    tabelle = {}
    table_counter = 0

    figure_with_tables = tree.xpath('//figure[.//table]')

    for figure in figure_with_tables:
        try:
            try:
                table_id = figure.xpath("@id")[0]
            except:
                # cerca l'id al secondo div padre perché a volte è salvato lì
                table_id = figure.xpath('ancestor::div[2]/@id')[0]
            table = figure.xpath('.//table')[0]
            table_counter += 1
            table_key = f"id_table_{table_counter}"

            caption = figure.xpath('.//figcaption//text()')
            caption_text = '' 

            if caption:
                caption_text = ''.join([c.strip() for c in caption]).replace('  ', ' ')

            dati_tabella = []
            rows = table.xpath('.//tr[position()>1]')
            for row in rows:
                cols = row.xpath('.//td')
                dati_row = [etree.tostring(col, encoding='unicode', method='html') for col in cols]
                dati_tabella.append(dati_row)

            references = tree.xpath(f"//p[a/@href = '#{table_id}']")
            references_text = [ref.xpath('string(.)').replace('\n', '').strip() for ref in references]   #elimina /n ma introduce spazi bianchi
            
            note_a_pie_di_pagina = []
            footnotes = figure.xpath(".//span[contains(@id, 'footnote') and contains(@class, 'ltx_text')]")
            for footnote in footnotes:
                logging.info(f"footnote found: {footnote} at table: {table_id}")

                footnote_text = " ".join(footnote.xpath(".//text()")).strip()
                if footnote_text and footnote_text != " ":
                    footnote_text = footnote_text.replace("\n", " ")
                    note_a_pie_di_pagina.append(footnote_text)

            tabelle[table_key] = {
                "caption": caption_text,
                "table": dati_tabella,
                "footnotes": note_a_pie_di_pagina,
                "references": references_text,
            }

        except Exception as e:
            logging.error(f"figure info: {figure.getchildren()}")
            logging.error(f"Error processing figure in {file_path}: {e.with_traceback()}")
            logging.error(f"Figure content: {etree.tostring(figure, encoding='unicode', pretty_print=True)}")
            break

    # for table_key, table_data in tabelle.items():
    #     logging.info(f"Footnotes for table {table_key}: {table_data['footnotes']}")
    return tabelle

In [95]:
# Funzione per salvare i dati estratti in un file JSON
def salva_dati_in_json(dati_estratti, article_id, extraction_path):
    tabelle = dati_estratti

    # Salva i dati in un file JSON
    output_file_path = f"{extraction_path}/{article_id}.json"
    os.makedirs(extraction_path, exist_ok=True)
    with open(output_file_path, 'w', encoding='utf-8') as json_file:
        json.dump(tabelle, json_file, ensure_ascii=False, indent=4)

    #logging.info(f"Dati salvati in {output_file_path}")

In [89]:
# per testare su di un unico file
file_path = os.path.join(SOURCES_DIR, 'ar5iv_article_2406.14952.html')

dati_estratti = estrai_dati_da_file(file_path)
logging.info(dati_estratti)
# Salva i dati in JSON
salva_dati_in_json(dati_estratti, '0', EXTRACTION_DIR)

2024-10-17 00:52:47,426 - INFO - figure: <Element figure at 0x7fd49dad9040>
2024-10-17 00:52:47,428 - INFO - table_id:S2.T1
2024-10-17 00:52:47,433 - INFO - figure: <Element figure at 0x7fd49dada640>
2024-10-17 00:52:47,434 - INFO - table_id:S3.T2
2024-10-17 00:52:47,440 - INFO - figure: <Element figure at 0x7fd49dad8680>
2024-10-17 00:52:47,440 - INFO - table_id:S3.T3
2024-10-17 00:52:47,447 - INFO - figure: <Element figure at 0x7fd49dad9800>
2024-10-17 00:52:47,448 - INFO - table_id:S3.T4
2024-10-17 00:52:47,453 - INFO - figure: <Element figure at 0x7fd49dad8f00>
2024-10-17 00:52:47,454 - INFO - table_id:A1.T5
2024-10-17 00:52:47,458 - INFO - figure: <Element figure at 0x7fd49dadb380>
2024-10-17 00:52:47,459 - INFO - table_id:A1.T6
2024-10-17 00:52:47,467 - INFO - figure: <Element figure at 0x7fd49dadbb80>
2024-10-17 00:52:47,469 - INFO - table_id:A1.SS3.tab1
2024-10-17 00:52:47,475 - INFO - figure: <Element figure at 0x7fd49dadaa40>
2024-10-17 00:52:47,477 - INFO - table_id:A1.T8
20

sto cercando le tabelle..
[<Element figure at 0x7fd49dad9040>, <Element figure at 0x7fd49dada640>, <Element figure at 0x7fd49dad8680>, <Element figure at 0x7fd49dad9800>, <Element figure at 0x7fd49dad8f00>, <Element figure at 0x7fd49dadb380>, <Element figure at 0x7fd49dadbb80>, <Element figure at 0x7fd49dadaa40>, <Element figure at 0x7fd49dada240>, <Element figure at 0x7fd49dad8940>, <Element figure at 0x7fd49dadb4c0>, <Element figure at 0x7fd49dadb500>, <Element figure at 0x7fd49dadae00>, <Element figure at 0x7fd49dad9a00>, <Element figure at 0x7fd49dada680>, <Element figure at 0x7fd49dadbc80>]


In [96]:
# Itera attraverso i file nella cartella 'articoli' e estrae le tabelle, didascalie, note e riferimenti
total_tables = 0
total_captions = 0
total_footnotes = 0
total_references = 0
articolo_counter = 0

#questo é perché nel salvataggio dei file ho creato cartella articoli dentro a sources
for filename in os.listdir(SOURCES_DIR):
    file_path = os.path.join(SOURCES_DIR, filename)
    article_id = filename.split('_')[2].replace('.html', '')  # Estrae l'ID dal nome del file (primo elemento prima del punto)

    articolo_counter += 1  # Incrementa il contatore
    logging.info(f"Estraendo dati dall'articolo {articolo_counter}: {filename}...")

    # Estrai i dati dal file HTML
    dati_estratti = estrai_dati_da_file(file_path)

    # Update statistics counters
    for table_data in dati_estratti.values():
        total_tables += 1
        if table_data["caption"]:
            total_captions += 1
        total_footnotes += len(table_data["footnotes"])
        total_references += len(table_data["references"])

    # Salva i dati in JSON
    salva_dati_in_json(dati_estratti, article_id, EXTRACTION_DIR)
        
# Print statistics
logging.info("\n--- Extraction Statistics ---")
logging.info(f"Total Articles Processed: {articolo_counter}")
logging.info(f"Total Tables Found: {total_tables}")
logging.info(f"Total Captions Found: {total_captions}")
logging.info(f"Total Footnotes Found: {total_footnotes}")
logging.info(f"Total References Found: {total_references}")

2024-10-17 00:56:39,820 - INFO - Estraendo dati dall'articolo 1: ar5iv_article_2401.17749.html...
2024-10-17 00:56:39,831 - INFO - Estraendo dati dall'articolo 2: ar5iv_article_2401.02500.html...


2024-10-17 00:56:39,894 - INFO - Estraendo dati dall'articolo 3: ar5iv_article_2305.15021.html...
2024-10-17 00:56:39,944 - INFO - Estraendo dati dall'articolo 4: ar5iv_article_2402.14871.html...
2024-10-17 00:56:39,960 - INFO - Estraendo dati dall'articolo 5: ar5iv_article_2406.06211.html...
2024-10-17 00:56:40,008 - INFO - Estraendo dati dall'articolo 6: ar5iv_article_2406.16273.html...


sto cercando le tabelle..
[]
sto cercando le tabelle..
[<Element figure at 0x7fd49da18180>]
sto cercando le tabelle..
[<Element figure at 0x7fd49d926040>, <Element figure at 0x7fd49daf5000>, <Element figure at 0x7fd49daf7a80>]
sto cercando le tabelle..
[<Element figure at 0x7fd49da6d380>, <Element figure at 0x7fd49da6e080>, <Element figure at 0x7fd49da6d740>, <Element figure at 0x7fd49da6d140>]
sto cercando le tabelle..
[<Element figure at 0x7fd49db57400>, <Element figure at 0x7fd49db564c0>, <Element figure at 0x7fd49d7a5800>, <Element figure at 0x7fd49d7a7640>, <Element figure at 0x7fd49d7a4840>, <Element figure at 0x7fd49d7a5d40>, <Element figure at 0x7fd49d7a5540>]


2024-10-17 00:56:40,065 - INFO - Estraendo dati dall'articolo 7: ar5iv_article_2306.06687.html...
2024-10-17 00:56:40,115 - INFO - Estraendo dati dall'articolo 8: ar5iv_article_2407.10022.html...
2024-10-17 00:56:40,137 - INFO - Estraendo dati dall'articolo 9: ar5iv_article_2311.09510.html...
2024-10-17 00:56:40,149 - INFO - Estraendo dati dall'articolo 10: ar5iv_article_2310.16301.html...
2024-10-17 00:56:40,184 - INFO - Estraendo dati dall'articolo 11: ar5iv_article_2407.17086.html...
2024-10-17 00:56:40,193 - INFO - Estraendo dati dall'articolo 12: ar5iv_article_2401.16107.html...
2024-10-17 00:56:40,228 - INFO - Estraendo dati dall'articolo 13: ar5iv_article_2404.04752.html...


sto cercando le tabelle..
[<Element figure at 0x7fd49db56d40>, <Element figure at 0x7fd49d924780>]
sto cercando le tabelle..
[<Element figure at 0x7fd49d924780>, <Element figure at 0x7fd49d926040>, <Element figure at 0x7fd49d926880>, <Element figure at 0x7fd49da43c40>, <Element figure at 0x7fd49da6e800>]
sto cercando le tabelle..
[<Element figure at 0x7fd49db4d140>]
sto cercando le tabelle..
[<Element figure at 0x7fd49d7ab4c0>, <Element figure at 0x7fd49d8b5c00>]
sto cercando le tabelle..
[<Element figure at 0x7fd49db55e00>, <Element figure at 0x7fd49db87800>, <Element figure at 0x7fd49d7a4c80>, <Element figure at 0x7fd49d7a7640>, <Element figure at 0x7fd49d7a4840>]
sto cercando le tabelle..
[]
sto cercando le tabelle..
[<Element figure at 0x7fd49fd692c0>, <Element figure at 0x7fd49fd68500>, <Element figure at 0x7fd49fd6a2c0>, <Element figure at 0x7fd49fd69740>]
sto cercando le tabelle..
[]


...

sto cercando le tabelle..
[<Element figure at 0x7fd49d7b97c0>, <Element figure at 0x7fd49d7b9680>, <Element figure at 0x7fd49d7b9940>]
sto cercando le tabelle..
[<Element figure at 0x7fd49d5c5d80>, <Element figure at 0x7fd49d5c5240>, <Element figure at 0x7fd49d5c6640>, <Element figure at 0x7fd49d5c7900>]
sto cercando le tabelle..
[<Element figure at 0x7fd49d5c5940>, <Element figure at 0x7fd49d5c58c0>, <Element figure at 0x7fd49d5c6180>]
sto cercando le tabelle..
[]
sto cercando le tabelle..
[<Element figure at 0x7fd49d7c8a80>, <Element figure at 0x7fd49d7c83c0>, <Element figure at 0x7fd49d7ca240>, <Element figure at 0x7fd49d7c9740>, <Element figure at 0x7fd49d7ca4c0>, <Element figure at 0x7fd49d7cbcc0>, <Element figure at 0x7fd49d7c8580>, <Element figure at 0x7fd49d7c8680>, <Element figure at 0x7fd49d7cb980>, <Element figure at 0x7fd49d7c96c0>, <Element figure at 0x7fd49d7c8e00>, <Element figure at 0x7fd49d7c82c0>, <Element figure at 0x7fd49d7cb8c0>, <Element figure at 0x7fd49d7c8440>,

2024-10-17 00:57:12,292 - INFO - Estraendo dati dall'articolo 361: ar5iv_article_2310.14201.html...
2024-10-17 00:57:12,340 - INFO - Estraendo dati dall'articolo 362: ar5iv_article_2305.13712.html...
2024-10-17 00:57:12,391 - INFO - Estraendo dati dall'articolo 363: ar5iv_article_2312.17025.html...
2024-10-17 00:57:12,419 - INFO - Estraendo dati dall'articolo 364: ar5iv_article_2405.15145.html...
2024-10-17 00:57:12,524 - INFO - Estraendo dati dall'articolo 365: ar5iv_article_2406.19859.html...


sto cercando le tabelle..
[]
sto cercando le tabelle..
[<Element figure at 0x7fd49fce70c0>, <Element figure at 0x7fd49d5ac1c0>, <Element figure at 0x7fd49d5ade00>, <Element figure at 0x7fd49d5afbc0>, <Element figure at 0x7fd49d5af100>, <Element figure at 0x7fd49d5ada80>]
sto cercando le tabelle..
[<Element figure at 0x7fd49d5af740>]
sto cercando le tabelle..
[<Element figure at 0x7fd49d5ac280>, <Element figure at 0x7fd49d5afd80>, <Element figure at 0x7fd49d5add00>, <Element figure at 0x7fd49d5ae240>, <Element figure at 0x7fd49d5ade40>, <Element figure at 0x7fd49d5aedc0>, <Element figure at 0x7fd49d5afb40>, <Element figure at 0x7fd49d5ace00>, <Element figure at 0x7fd49d5ad200>]


2024-10-17 00:57:12,589 - INFO - Estraendo dati dall'articolo 366: ar5iv_article_2405.11403.html...
2024-10-17 00:57:12,634 - INFO - 
--- Extraction Statistics ---
2024-10-17 00:57:12,635 - INFO - Total Articles Processed: 366
2024-10-17 00:57:12,638 - INFO - Total Tables Found: 1879
2024-10-17 00:57:12,639 - INFO - Total Captions Found: 1859
2024-10-17 00:57:12,640 - INFO - Total Footnotes Found: 30
2024-10-17 00:57:12,641 - INFO - Total References Found: 2051


sto cercando le tabelle..
[<Element figure at 0x7fd49d5ae900>, <Element figure at 0x7fd49d5afb80>]
sto cercando le tabelle..
[<Element figure at 0x7fd49d8abe00>, <Element figure at 0x7fd49d933900>, <Element figure at 0x7fd49d932100>, <Element figure at 0x7fd49d932dc0>, <Element figure at 0x7fd49d899b40>, <Element figure at 0x7fd49db55cc0>, <Element figure at 0x7fd49d5ad080>, <Element figure at 0x7fd49d5ae500>]
